In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import json
import os
import pandas as pd

In [3]:
! pip install ijson

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 133 kB 4.0 MB/s 


In [4]:
import ijson 
from itertools import islice

In [5]:
path_to_data = '/content/drive/MyDrive/advNLP/twibot22/'
tweet_files = [x for x in os.listdir(path_to_data) if ('tweet' in x) and ('processed' not in x)]

In [6]:
# for tweet_file in tweet_files:

#   print(tweet_file)

#   tweet_file_path = os.path.join(path_to_data, tweet_file)
#   data = []

#   with open(tweet_file_path, 'r') as f:
#     objects = ijson.items(f, 'item')

#     x = next(objects, 0)

#     while x!=0:
#       data.append({"author_id" : x["author_id"], "id" : x["id"], "text" : x["text"]})
#       x = next(objects, 0)

#   new_file_path = os.path.join(path_to_data, "processed_"+tweet_file)
#   with open(new_file_path, 'w') as f:
#     json.dump(data, f)

#   del data 
  


In [7]:
with open(os.path.join(path_to_data, 'user.json'), 'r') as f:
  user_data = json.loads(f.read())

In [8]:
import pandas as pd

data = pd.read_csv(os.path.join(path_to_data, 'compiled_nopost.csv'))
data.set_index('id', inplace=True)

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (3,4) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [9]:
data['following'] = data['following'].apply(lambda x: '[]' if pd.isnull(x) else x)
data['followers'] = data['followers'].apply(lambda x: '[]' if pd.isnull(x) else x)

In [10]:
split_names = data['split'].unique()
data = data.to_dict(orient='index')
uids = list(data.keys())

In [11]:
while len(user_data)>0:
  user_datum = user_data.pop()
  uid = user_datum['id']
  for key in user_datum:
    data[uid][key] = user_datum[key]


In [12]:
proc_tweet_files = [x for x in os.listdir(path_to_data) if ('tweet' in x) and ('processed' in x)]

In [13]:
for tweet_file in proc_tweet_files:
  print(tweet_file)
  tweet_file_path = os.path.join(path_to_data, tweet_file)
  with open(tweet_file_path, 'r') as f:
    objects = ijson.items(f, 'item')
    x = next(objects, 0)
    while x!=0:
      uid = 'u' + str(x['author_id'])
      data[uid]['uid'] = uid

      if 'tweets' not in data[uid]:
        data[uid]['tweets'] = []
        
      data[uid]['tweets'].append(x['text'])
      x = next(objects, 0)

processed_tweet_0.json
processed_tweet_1.json
processed_tweet_2.json
processed_tweet_3.json
processed_tweet_4.json
processed_tweet_5.json
processed_tweet_6.json
processed_tweet_7.json
processed_tweet_8.json


In [14]:
split_data = {split_name : [] for split_name in split_names}

for uid in uids:
  split_name = data[uid]['split']
  split_data[split_name].append(data.pop(uid))


In [15]:
for split_name in split_data:
  with open(os.path.join(path_to_data, 'processed', f'processed_{split_name}.json'), 'w') as f:
    json.dump(split_data[split_name], f)